##VARIOUS OPTIONS FOR INTERPOLATION ON LAT/LON GRID

In [1]:
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import urllib
import netCDF4
from mpl_toolkits.basemap import Basemap
import scipy.interpolate
import sys

from scipy import ndimage

In [2]:
from IPython.html.widgets import *


:0: FutureWarning: IPython widgets are experimental and may change in the future.


For testing purposes we are interpolating on a lat/lon grid the bathymetry taken from SRTV30v6.

In [3]:
minlat = 28.
maxlat = 48.
minlon = -12.
maxlon = 48.
isub=2


In [4]:
# Read data from: http://coastwatch.pfeg.noaa.gov/erddap/griddap/usgsCeSrtm30v6.html
# using the netCDF output option
 
base_url='http://coastwatch.pfeg.noaa.gov/erddap/griddap/usgsCeSrtm30v6.nc?'
query='topo[(%f):%d:(%f)][(%f):%d:(%f)]' % (maxlat,isub,minlat,minlon,isub,maxlon)
url = base_url+query
print url


http://coastwatch.pfeg.noaa.gov/erddap/griddap/usgsCeSrtm30v6.nc?topo[(48.000000):2:(28.000000)][(-12.000000):2:(48.000000)]


In [6]:
# store data in NetCDF file
file='usgsCeSrtm30v6.nc'
urllib.urlretrieve (url, file)



('usgsCeSrtm30v6.nc', <httplib.HTTPMessage instance at 0x4085d40>)

In [7]:
# open NetCDF data in
nc = netCDF4.Dataset(file)
ncv = nc.variables
print ncv.keys()

lon = ncv['longitude'][:]
lat = ncv['latitude'][:]
lons, lats = np.meshgrid(lon,lat)
topo = ncv['topo'][:,:]


[u'latitude', u'longitude', u'topo']


In [8]:
print topo.shape

(1201, 3601)


In [ ]:
# Create map
m = Basemap(projection='cyl', llcrnrlat=minlat,urcrnrlat=maxlat,llcrnrlon=minlon, urcrnrlon=maxlon,resolution='l')
fig = plt.figure(figsize=(10,8))
cs = m.pcolormesh(lons,lats,topo,cmap=plt.cm.jet,latlon=True)
m.drawcoastlines()
m.drawmapboundary()
plt.title('SMRT30 - Bathymetry/Topography')
cbar = plt.colorbar(orientation='horizontal', extend='both')
cbar.ax.set_xlabel('meters')

plt.show()

###define the new grid to interpolate

In [10]:
ni=70
nj=30

In [11]:
x=np.linspace(minlon,maxlon,ni,endpoint=True)
print x.shape
y=np.linspace(minlat,maxlat,nj,endpoint=True)
print y.shape

(70,)
(30,)


In [12]:
xx,yy=np.meshgrid(x,y)

##Interpolation using basemap tool

Because for this function the arrays must be increasing we flip lat 

In [13]:
mtopo,bxx,byy = \
  m.transform_scalar(np.flipud(topo),lon,np.flipud(lat),ni,nj,returnxy=True,masked=True,order=3)  # we flip the lat to make it increasing

In [14]:
fig = plt.figure(figsize=(10,8))
cs = m.pcolormesh(bxx,byy,mtopo,cmap=plt.cm.jet,latlon=True)
m.drawcoastlines()
m.drawmapboundary()
plt.title('basemap transform')
cbar = plt.colorbar(orientation='horizontal', extend='both')
cbar.ax.set_xlabel('meters')

plt.show()

In [16]:
def bmap():
 mtopo,bxx,byy = \
   m.transform_scalar(np.flipud(topo),lon,np.flipud(lat),ni,nj,returnxy=True,masked=True)  # we flip the lat to make it increasing 
 return mtopo,bxx,byy

In [17]:
%timeit var,la,lo=bmap()

100 loops, best of 3: 8.68 ms per loop


##Interpolation using RectBivariateSpline

Because for this function the arrays must be increasing we flip lat 

In [21]:
blat=np.flipud(lat)
btopo=np.flipud(topo) # since we flip lat

In [30]:
sol=scipy.interpolate.RectBivariateSpline(lon,blat,btopo.T) # note the transpose btopo

In [31]:
itopo=[]
for x,y in zip(xx.ravel(),yy.ravel()):
      itopo.append(sol(x,y).ravel()[0])

itopo=np.array(itopo)
itopo=itopo.reshape(xx.shape)


In [32]:
fig = plt.figure(figsize=(10,8))
cs = m.pcolormesh(xx,yy,itopo,cmap=plt.cm.jet,latlon=True)
m.drawcoastlines()
m.drawmapboundary()
plt.title('RectBivariateSpline')
cbar = plt.colorbar(orientation='horizontal', extend='both')
cbar.ax.set_xlabel('meters')

plt.show()

In [33]:
def rbs():
    sol=scipy.interpolate.RectBivariateSpline(lon,np.flipud(lat),np.flipud(topo).T) # note the transpose btopo
    itopo=[]
    for x,y in zip(xx.ravel(),yy.ravel()):
      itopo.append(sol(x,y).ravel()[0])

    itopo=np.array(itopo)
    itopo=itopo.reshape(xx.shape)
    return itopo,xx,yy

In [34]:
%timeit var,la,lo=rbs()

1 loops, best of 3: 911 ms per loop


In [35]:
dx=np.abs(xx-bxx).max()
print dx

3.81469726562e-06


In [36]:
dy=np.abs(yy-byy).max()
print dy

2.63082569774e-06


In [37]:
dv=np.abs(itopo-mtopo).max()
print dv

0.625348504378


In [38]:
np.abs(itopo-mtopo).mean()

0.24098741638504972

##Interpolation using RegularGridInterpolator

In [ ]:
# compute the new grid location in the original grid



In [39]:
myf = scipy.interpolate.RegularGridInterpolator((lon, blat), btopo.T,method='linear')

In [40]:
rtopo=[]
for x,y in zip(xx.ravel(),yy.ravel()):
      rtopo.append(myf([x,y]).ravel()[0])

rtopo=np.array(rtopo)
rtopo=rtopo.reshape(xx.shape)


In [41]:
def rgi():
    myf = scipy.interpolate.RegularGridInterpolator((lon, blat), btopo.T)
    rtopo=[]
    for x,y in zip(xx.ravel(),yy.ravel()):
      rtopo.append(myf([x,y]).ravel()[0])

    rtopo=np.array(rtopo)
    rtopo=rtopo.reshape(xx.shape)
    return rtopo,xx,yy

In [42]:
%timeit var,la,lo=rgi()

1 loops, best of 3: 987 ms per loop


In [43]:
dv=np.abs(rtopo-mtopo).max()
print dv

162.886056972


In [45]:
#plot
var,la,lo=rgi()
fig = plt.figure(figsize=(10,8))
cs = m.pcolormesh(la,lo,var,cmap=plt.cm.jet,latlon=True)
m.drawcoastlines()
m.drawmapboundary()
plt.title('Interpolation with RegularGridInterpolator')
cbar = plt.colorbar(orientation='horizontal', extend='both')
cbar.ax.set_xlabel('meters')

plt.show()

##Interpolation using ndimage

In [96]:
xd=np.arange(lon.shape[0],ni)
xd

array([], dtype=int64)

In [97]:
lon.shape[0]

3601

In [50]:
import matplotlib
matplotlib.get_backend()

'module://IPython.kernel.zmq.pylab.backend_inline'